In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spacy
spacy.load('en_core_web_sm')
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import GeonameAnnotator
from epitator.resolved_keyword_annotator import ResolvedKeywordAnnotator
from epitator.count_annotator import CountAnnotator
from epitator.date_annotator import DateAnnotator
from itertools import groupby
import datetime
import sys
import pandas as pd
import re
import numpy as np
import epitator
import numpy as np

In [3]:
import pandas as pd
import pickle
import os
from who_scraper import scrape
from my_utils import extract_from_url, flatten_list, get_edb
from annotator import *
from wiki_country_parser import get_wiki_countries_df

Pickle not found. Start scraping..
...scraping complete.


# WHO DF

In [4]:
path = os.path.join("pickles","parsed_who_df.p")

In [ ]:
all_links = scrape(years=['2018'],proxies=None)
extracted = extract_from_url(all_links)

In [ ]:
parsed_whos_df = pd.DataFrame.from_dict(create_annotated_database(extracted, entity_funcs_and_params=[
                                                                      (geonames, {"raw":True}),
                                                                      (cases, {"raw":True}),
                                                                      (dates, {"raw":True}),
                                                                      keywords]))

In [ ]:
pickle.dump(parsed_whos_df,open(path,"wb"))

In [5]:
parsed_whos_df = pickle.load(open(path,"rb"))
parsed_whos_df.head()

,texts,dates,cases,keywords,geonames
0,Ebola virus disease – Democratic Republic of t...,"[2018-11-27, 2018-10-01, 2018-01-01, 2018-11-2...","[36, 13, 9, 6, 6, 1, 1, 2, 42, 12, 6, 6, 125, ...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, South Sudan..."
1,Ebola virus disease – Democratic Republic of t...,"[2018-11-16, 2018-11-01, 1920-11-14, 2018-11-2...","[18, 36, 7, 6, 1, 1, 1, 5, 39, 10, 6, 4, 113, ...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, Democratic ..."
2,Middle East respiratory syndrome coronavirus (...,"[2018-11-01, 2018-10-16, 2018-10-30, 2005-01-01]","[4, 1, 1, 2266, 804, 1]",[Middle East respiratory syndrome],"[Middle East, Kingdom of Saudi Arabia, Kingdom..."
3,Ebola virus disease – Democratic Republic of t...,"[2018-11-01, 2018-11-07, 2018-11-13, 2018-11-0...","[2, 1, 31, 4, 3, 3, 3, 31, 12, 9, 2, 103, 14, ...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, Democratic ..."
4,Ebola virus disease – Democratic Republic of t...,"[2018-11-06, 2018-11-06, 2018-11-06, 2018-10-3...","[300, 31, 29, 15, 7, 4, 2, 2, 3, 28, 10, 88, 3...",[Ebola hemorrhagic fever],"[Democratic Republic of the Congo, Democratic ..."


# EDB Statistics

In [6]:
edb = get_edb()

/Users/aussabbood/github/nlp-surveillance/edb_clean.py:39: UserWarning: "31.09.2018" is not an existing date. Please change it
  warnings.warn('"{}" is not an existing date. Please change it'.format(date))


# CONTINUE HERE

## Country statistic

In [8]:
print(len(edb)," entries", "in edb")
countries_not_null = edb[pd.notnull(edb["Ausgangs- bzw. Ausbruchsland"])]
print(len(countries_not_null)," are not empty")
unique = set(c[0] if type(c)==list else c for c in countries_not_null['Ausgangs- bzw. Ausbruchsland'].tolist())
print(len(unique), " are unique")

295  entries in edb
280  are not empty
80  are unique


In [9]:
wiki_countries_df = get_wiki_countries_df()
same_word_translated = []
for _, row in wiki_countries_df.iterrows():
    if row['state_name_de'] == row['translation_state_name']:
        same_word_translated.append(row['state_name_de'])
    elif row['full_state_name_de'] == row['translation_state_name']:
        same_word_translated.append(row['full_state_name_de'])
unprocesed_edb = pd.read_csv("edb.csv", sep=";")

# Exlude words that are unchanged compared to unproccessed version
unchanged_country_names = [e for e in unique if e in unprocesed_edb['Ausgangs- bzw. Ausbruchsland '].tolist()]
# Exlude words that are the same after translation
unchanged_country_names = [e for e in unchanged_country_names if e not in same_word_translated]
# Exlude words that were English in the first place
unchanged_country_names =  [e for e in unchanged_country_names if e not in wiki_countries_df["translation_state_name"].tolist()]
print("{} countries were not translated".format(len(unchanged_country_names)))
unchanged_country_names

8 countries were not translated


['Taiwan',
 'Französisch-Polynesien',
 'VAE Dubai',
 'La Reunion',
 'Französiche Guyana',
 'Nordeuropa',
 'DRCongo',
 'Typhus']

## Direct comparison to source

In [10]:
indices_sources = list(range(15,26))
indices_sources_and_countries = indices_sources
indices_sources_and_countries.extend([3,9,11,12])
indices_sources_and_countries

[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 3, 9, 11, 12]

In [11]:
sources = edb.iloc[:,indices_sources_and_countries].copy(deep=True) # Get only the columns mentioning sources

In [12]:
mask = sources.copy(deep=True) # Create a mask for filtering
# Use only the columns about sources to filter
for column in sources.columns[:11]:
    mask[column] = sources[column].str.contains('who',na=False) # Extract all the entries that have the word "who"
    mask[column] = sources[column].str.contains('don',na=False) # and "don"
indices = [i for i in range(len(mask)) if not mask.iloc[i,:11].any()] 
sources_filtered = sources.drop(np.reshape(indices,(len(indices),)))# Drop all rows that don't mention "who" or "don"

In [13]:
# Only get the necessary information from the who dons for comparison
comparable_who = parsed_whos_df.iloc[:,[1,2,4]].copy(deep=True)

In [14]:
dates = sources_filtered.iloc[:,[1,4,7,10]].copy(deep=True)

In [15]:
dates = dates.replace("nan",np.nan)

In [16]:
# Clean dates
dates = dates.values.tolist()
dates = [list(filter(lambda x: x==x, row)) for row in dates]
dates

[['2018-03-15'],
 ['2018-04-13'],
 ['2018-04-15', '2018-04-20', '2018-04-18'],
 ['2018-05-11', '2018-05-10'],
 ['2018-05-15', '2018-05-14'],
 ['2018-05-17', '2018-05-17'],
 ['2018-05-22', '2018-05-21'],
 ['2018-05-31'],
 ['2018-06-05', '2018-06-05'],
 ['2018-06-12'],
 ['2018-06-18'],
 ['2018-07-30', '2018-08-04'],
 ['2018-08-07', '2018-06-25'],
 ['2018-08-07'],
 ['2018-07-02'],
 ['2018-08-24'],
 ['2018-08-31']]

In [17]:
# Clean faelle
faelle = sources_filtered.iloc[:,-3:].copy(deep=True)
faelle = faelle.replace(np.nan,"nan")
faelle = faelle.values.tolist()
faelle = [list(filter(lambda x: x==x, row)) for row in faelle]
faelle

[['nan', 'nan', 'nan'],
 ['20', '19', 1.0],
 ['nan', '413', 9.0],
 ['34', '2', 'nan'],
 ['41', '2', 'nan'],
 ['44', '3', 'nan'],
 ['46', 'nan', 'nan'],
 ['nan', '15', 'nan'],
 ['16', '1', 'nan'],
 ['995', '114', 'nan'],
 ['nan', 'nan', 'nan'],
 ['nan', '16', 27.0],
 ['3', '3', 'nan'],
 ['nan', 'nan', 'nan'],
 ['nan', '1, aber 2 Nachweise bei asymptomatischen Kindern', 'nan'],
 ['1', '1', 'nan'],
 ['1', '1', 'nan']]

In [18]:
cases = []
for row in faelle:
    strings = []
    if len(row) >=1 :
        for string in row:
            string = str(string)
            found = re.findall(r"(\d{1,})",string)
            strings.extend([digit for digit in found if digit != '' and digit != "0"])
        cases.append(strings)
    else:
        cases.append(row)
cases

[[],
 ['20', '19', '1'],
 ['413', '9'],
 ['34', '2'],
 ['41', '2'],
 ['44', '3'],
 ['46'],
 ['15'],
 ['16', '1'],
 ['995', '114'],
 [],
 ['16', '27'],
 ['3', '3'],
 [],
 ['1', '2'],
 ['1', '1'],
 ['1', '1']]

In [19]:
sources_filtered.columns[-4]

'Ausgangs- bzw. Ausbruchsland'

In [20]:
comparable_edb = sources_filtered['Ausgangs- bzw. Ausbruchsland'].copy(deep=True).to_frame()
comparable_edb["cases"] = cases
comparable_edb["dates"] = dates
comparable_edb

,Ausgangs- bzw. Ausbruchsland,cases,dates
25,Oman,[],[2018-03-15]
52,Australia,"[20, 19, 1]",[2018-04-13]
70,Nigeria,"[413, 9]","[2018-04-15, 2018-04-20, 2018-04-18]"
107,Democratic Republic of the Congo,"[34, 2]","[2018-05-11, 2018-05-10]"
112,Democratic Republic of the Congo,"[41, 2]","[2018-05-15, 2018-05-14]"
113,Democratic Republic of the Congo,"[44, 3]","[2018-05-17, 2018-05-17]"
120,Democratic Republic of the Congo,[46],"[2018-05-22, 2018-05-21]"
140,India,[15],[2018-05-31]
145,Cameroon,"[16, 1]","[2018-06-05, 2018-06-05]"
149,Brazil,"[995, 114]",[2018-06-12]


In [21]:
comparable_who.head()

,dates,cases,geonames
0,"[2018-11-27, 2018-10-01, 2018-01-01, 2018-11-2...","[36, 13, 9, 6, 6, 1, 1, 2, 42, 12, 6, 6, 125, ...","[Democratic Republic of the Congo, South Sudan..."
1,"[2018-11-16, 2018-11-01, 1920-11-14, 2018-11-2...","[18, 36, 7, 6, 1, 1, 1, 5, 39, 10, 6, 4, 113, ...","[Democratic Republic of the Congo, Democratic ..."
2,"[2018-11-01, 2018-10-16, 2018-10-30, 2005-01-01]","[4, 1, 1, 2266, 804, 1]","[Middle East, Kingdom of Saudi Arabia, Kingdom..."
3,"[2018-11-01, 2018-11-07, 2018-11-13, 2018-11-0...","[2, 1, 31, 4, 3, 3, 3, 31, 12, 9, 2, 103, 14, ...","[Democratic Republic of the Congo, Democratic ..."
4,"[2018-11-06, 2018-11-06, 2018-11-06, 2018-10-3...","[300, 31, 29, 15, 7, 4, 2, 2, 3, 28, 10, 88, 3...","[Democratic Republic of the Congo, Democratic ..."


In [23]:
# Match countries
found = 0
found_c = []
for state in comparable_edb["Ausgangs- bzw. Ausbruchsland"]:
    for i in comparable_who["geonames"]:
        if isinstance(state,str):
            if list(filter(lambda x: re.findall(state,x),i)):
                found_c.append(state)
                found += 1
                break

                    

In [24]:
found_c

['Oman',
 'Australia',
 'Nigeria',
 'Democratic Republic of the Congo',
 'Democratic Republic of the Congo',
 'Democratic Republic of the Congo',
 'Democratic Republic of the Congo',
 'India',
 'Cameroon',
 'Brazil',
 'Kenya',
 'Democratic Republic of the Congo',
 'Papua New Guinea',
 'India',
 'United Kingdom']

In [25]:
[i for i in comparable_edb["Ausgangs- bzw. Ausbruchsland"] if i not in found_c]

[None, 'Französiche Guyana']

In [56]:
# Match dates
from my_utils import matching_elements
found_date = {}
for index, dates in enumerate(comparable_edb["dates"]):
    for i in comparable_who["dates"]:
        matches = matching_elements(dates,i)
        if len(matching_elemnts(dates,i)) >= 1:
            found_date[index] = matches        

In [53]:
found_date

{0: ['2018-03-15'],
 2: ['2018-04-15'],
 3: ['2018-05-10'],
 4: ['2018-05-15', '2018-05-14'],
 5: ['2018-05-17', '2018-05-17'],
 6: ['2018-05-21'],
 7: ['2018-05-31'],
 8: ['2018-06-05', '2018-06-05'],
 9: ['2018-06-12'],
 10: ['2018-06-18'],
 11: ['2018-07-30'],
 12: ['2018-08-07'],
 13: ['2018-08-07'],
 15: ['2018-08-24'],
 16: ['2018-08-31']}

In [57]:
found_cases = {}
for index, cases in enumerate(comparable_edb["cases"]):
    for i in comparable_who["cases"]:
        matches = matching_elements(cases,i)
        if len(matching_elemnts(dates,i)) >= 1:
            found_cases[index] = matches  

In [59]:
comparable_edb["cases"]

25              []
52     [20, 19, 1]
70        [413, 9]
107        [34, 2]
112        [41, 2]
113        [44, 3]
120           [46]
140           [15]
145        [16, 1]
149     [995, 114]
163             []
200       [16, 27]
202         [3, 3]
204             []
224         [1, 2]
247         [1, 1]
257         [1, 1]
Name: cases, dtype: object

In [26]:
def find_matches(df1_and_indices, df2_and_indices):
    indices_matches = []
    matches = []
    df1, columns1 = df1_and_indices
    df2, columns2 = df2_and_indices
    for index , row in df1.iterrows():
        for _, row in df2.iterrows():
            if set()

SyntaxError: invalid syntax (<ipython-input-26-73a8da0f66bc>, line 8)

In [ ]:
sources_filtered.columns[-2::-3]

In [ ]:
# Idea which is not correct and not complete
date_matches = {}
for column in sources_filtered.columns[-2::-3]: # Use only the columns mentioning dates
    date_matches[column] = [i for i in range(len(parsed_whos_df))\
                            if list(map(lambda x: str(x)[:-3],parsed_whos_df["date"].tolist()))[i] \
                            in list(map(lambda x: str(x)[:-12],sources_filtered[column].tolist()))]

In [ ]:
import itertools
indices_that_matched = list(set(itertools.chain(*date_matches.values())))

# To present

In [ ]:
ereignisdatenbank.iloc[sources_filtered.index.tolist(),[3,6,7,9]]

In [ ]:
parsed_whos_df.iloc[indices_that_matched,1:].sort_values("date")

In [ ]:
# Prettify the link description
link_description = [re.search(r'don/(.*)/en',all_links[i])[1]\
                    .replace('-', ' ',2).replace('-',', ',2).replace('-',' ')\
                    for i in range(len(all_links))]

In [ ]:
# Extract the most important columns
compare = parsed_whos_df.iloc[:,[1,3,4]].copy()
compare['link_description'] = pd.Series(link_description,index=compare.index)

In [ ]:
compare["date"].iloc[0]

In [ ]:
# These are the links that were faulty during annotation
to_check = compare[compare["keyword"].isnull()==True].index.values
links_to_check = np.asarray(all_links)[to_check] # Get the links that caused the bad annotations

In [ ]:
annotated_faulty_text = create_annotated_database(extract(links_to_check),raw=True)

In [ ]:
faulty_df = pd.DataFrame.from_dict(annotated_faulty_text).iloc[:,1:]
faulty_df

In [ ]:
# Entries that had the occurence of a geographical entity more than once
parsed_whos_df[parsed_whos_df["geoname"].str.len()>1]